In [ ]:
# pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
from matplotlib import pyplot as plt

In [2]:
mp_hollistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [16]:
# reaad video and save frames
cap = cv2.VideoCapture('00431.mp4') # video path
frameRate = cap.get(2) # frame rate

while(cap.isOpened()):
    frameId = cap.get(1) # current frame number
    ret, frame = cap.read() # read frame
    if ret != True:
        break
    if (frameId % ((int(frameRate)+1)*2) == 0): # save every 2 seconds
        # save to a folder
        filename = 'frames1/frame' + str(int(frameId)) + '.jpg'
        cv2.imwrite(filename, frame)


cap.release()
cv2.destroyAllWindows()